In [2]:
import sys
!{sys.executable} -m pip install essentia
!{sys.executable} -m pip install pymongo

In [ ]:
import os
import essentia
import essentia.standard as es
from pymongo import MongoClient

In [4]:
# connect to db
client = MongoClient('mongodb://mongo:27017/')
db = client.music_analysis
songs = db.songs

In [ ]:
ignorePrefix = [
    'lowlevel.barkbands',
    'lowlevel.dissonance',
    'lowlevel.erbbands',
    'lowlevel.melbands',
    'lowlevel.mfcc',
    'lowlevel.hfc',
    'lowlevel.loudness_ebu128',
    'lowlevel.pitch_salience',
    'lowlevel.silence_rate',
    'lowlevel.spectral_',
    'lowlevel.zerocrossingrate',
    'tonal.hpcp',
    'rhythm.beats_loudness',
    'rhythm.beats_position',
    'rhythm.bpm_histogram',
    'tonal.chords_histogram',
    'tonal.thpcp',
    'tonal.chords_strength',
    'tonal.tuning_diatonic_strength',
    'tonal.tuning_equal_tempered_deviation',
    'tonal.tuning_nontempered_energy_ratio',
    'tonal.key',
    'lowlevel.gfcc',
    'lowlevel.mfcc',
    'metadata'
]
ignoreSuffix = [
    '.probability',
    '.strength'
]
whitelist = [
    'lowlevel.dissonance.median',
    'rhythm.beats_loudness.median',
    'tonal.chords_strength.median',
    'tonal.key_krumhansl.key',
    'tonal.key_krumhansl.scale'
]

# scan all directories for audio files & store data
# approx 11k songs total
directories = [x[0] for x in os.walk('./../../music/')]
for directory in directories:
    for file in os.listdir(directory):
        if file.endswith('.mp3') or file.endswith('.m4a') or file.endswith('.ogg') or file.endswith('.m4p') or file.endswith('.flac'):
            filename = directory+'/'+file
            # check if already in db
            record = songs.find_one({'filename': filename})
            if record is None:
                # analyze song
                try:
                    features, features_frames = es.MusicExtractor(profile="profile.yml")(filename)
                except RuntimeError:
                    print("Can't analyze "+filename)
                    continue
                # make object to store in mongo
                song = {'filename': filename}
                for stat in features.descriptorNames():
                    if stat in whitelist or (not 'not_' in stat and not stat.startswith(tuple(ignorePrefix)) and not stat.endswith(tuple(ignoreSuffix))):
                        # remove dots from key
                        stat_cleaned = stat.replace('.','')
                        song[stat_cleaned] = features[stat]
                # store in database
                song_id = songs.insert_one(song).inserted_id
                print(filename+' - '+str(song_id))
            else:
                print("Skipping "+filename)

Skipping ./../../music/Gorillaz feat. Daley/Doncamatic/01 Doncamatic.mp3
Skipping ./../../music/Gorillaz feat. Daley/Doncamatic/02 Doncamatic (Instrumental).mp3
Skipping ./../../music/Themselves/The No Music of Aiff's_ The No Music Remixed/05 Poison Pit (Why_ remix).mp3
Skipping ./../../music/Themselves/The No Music of Aiff's_ The No Music Remixed/08 Dr. Moonorgun Please (Grapedope remix).mp3
Skipping ./../../music/Themselves/The No Music of Aiff's_ The No Music Remixed/06 Livetrap (Hood remix).mp3
Skipping ./../../music/Themselves/The No Music of Aiff's_ The No Music Remixed/11 Out in the Open (Notwist remix).mp3
Skipping ./../../music/Themselves/The No Music of Aiff's_ The No Music Remixed/09 Darkskydemo (Fog remix).mp3
Skipping ./../../music/Themselves/The No Music of Aiff's_ The No Music Remixed/01 Terror Fabulous.mp3
Skipping ./../../music/Themselves/The No Music of Aiff's_ The No Music Remixed/04 Good People Check (Hrvatski remix).mp3
Skipping ./../../music/Themselves/The No Musi

Skipping ./../../music/Samuel Truth/RISE/02 LevelUp.mp3
Skipping ./../../music/Samuel Truth/RISE/04 Mercury.mp3
Skipping ./../../music/Samuel Truth/RISE/03 Velvet.mp3
Skipping ./../../music/Portishead/Dummy/03 Strangers.mp3
Skipping ./../../music/Portishead/Dummy/07 Numb.mp3
Skipping ./../../music/Portishead/Dummy/04 It Could Be Sweet.mp3
Skipping ./../../music/Portishead/Dummy/05 Wandering Star.mp3
Skipping ./../../music/Portishead/Dummy/11 Glory Box.mp3
Skipping ./../../music/Portishead/Dummy/06 It’s a Fire.mp3
Skipping ./../../music/Portishead/Dummy/10 Biscuit.mp3
Skipping ./../../music/Portishead/Dummy/08 Roads.mp3
Skipping ./../../music/Portishead/Dummy/02 Sour Times.mp3
Skipping ./../../music/Portishead/Dummy/09 Pedestal.mp3
Skipping ./../../music/Portishead/PNYC/07 Over.mp3
Skipping ./../../music/Portishead/PNYC/06 Half Day Closing.mp3
Skipping ./../../music/Portishead/PNYC/03 All Mine.mp3
Skipping ./../../music/Portishead/PNYC/05 Only You.mp3
Skipping ./../../music/Portishead/P

Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/21 Kev Intro 99.mp3
Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/10 One for the Crime Scene, a Bullet for Your Time.mp3
Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/19 3 Sounds From 94.mp3
Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/03 Pase Rock's Preestyle.mp3
Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/15 Sao Paulo Arkansas.mp3
Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/20 Robot Snares Got No Cadence or Balance.mp3
Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/12 Diarrhea Takes Over Your Life.mp3
Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/11 Vikings Invade the Mediterranean but Don't Leave.mp3
Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/06 Between Man and Woman.mp3
Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/18 Wronge Posture.mp3
Skipping ./../../music/Prefuse 73/Extinguished_ Outtakes/13 I Can't Get My Eyes Off.mp

Skipping ./../../music/The Cinematic Orchestra/Man With a Movie Camera/17 All Things.mp3
Skipping ./../../music/The Cinematic Orchestra/Man With a Movie Camera/11 Theme de Yoyo.mp3
Skipping ./../../music/The Cinematic Orchestra/Every Day/07 Everyday.mp3
Skipping ./../../music/The Cinematic Orchestra/Every Day/04 Evolution.mp3
Skipping ./../../music/The Cinematic Orchestra/Every Day/03 Flite.mp3
Skipping ./../../music/The Cinematic Orchestra/Every Day/02 Burn Out.mp3
Skipping ./../../music/The Cinematic Orchestra/Every Day/05 Man With the Movie Camera.mp3
Skipping ./../../music/The Cinematic Orchestra/Every Day/01 All That You Give.mp3
Skipping ./../../music/The Cinematic Orchestra/Every Day/06 All Things to All Men.mp3
Skipping ./../../music/Super8 & Tab/Anjunabeats pres. Super8 & Tab 02/12 Perfect Day.mp3
Skipping ./../../music/Super8 & Tab/Anjunabeats pres. Super8 & Tab 02/04 Irufushi.mp3
Skipping ./../../music/Super8 & Tab/Anjunabeats pres. Super8 & Tab 02/11 Needs to Feel.mp3
Skipp

Skipping ./../../music/Jurassic 5/Feedback/07 Work It Out.mp3
Skipping ./../../music/Jurassic 5/Feedback/09 Get It Together.mp3
Skipping ./../../music/Jurassic 5/Feedback/10 Future Sound.mp3
Skipping ./../../music/Jurassic 5/Feedback/08 Where We At.mp3
Skipping ./../../music/Jurassic 5/Feedback/03 Brown Girl.mp3
Skipping ./../../music/Altra Volta string quartet/Water music/00 Water music. Based on improvisation - Triple concerto for faucet water pipes and fiddle.mp3
Skipping ./../../music/King Geedorah/Take Me to Your Leader/03 Krazy World.mp3
Skipping ./../../music/King Geedorah/Take Me to Your Leader/05 Monster Zero.mp3
Skipping ./../../music/King Geedorah/Take Me to Your Leader/06 Next Levels.mp3
Skipping ./../../music/King Geedorah/Take Me to Your Leader/10 Lockjaw.mp3
Skipping ./../../music/King Geedorah/Take Me to Your Leader/02 Fastlane.mp3
Skipping ./../../music/King Geedorah/Take Me to Your Leader/13 The Fine Print.mp3
Skipping ./../../music/King Geedorah/Take Me to Your Leade

...

./../../music/Sage Francis/Still Sickly Business/14 Damage 96 (live).mp3 - 5e51ad4f3f2129c97e160454
./../../music/Sage Francis/Still Sickly Business/12 Killing Time (live).mp3 - 5e51ad5e3f2129c97e160455
./../../music/Sage Francis/Still Sickly Business/02 Time of My Life Redux.mp3 - 5e51ad673f2129c97e160456
./../../music/Sage Francis/Still Sickly Business/19 Threewrite.mp3 - 5e51ad7c3f2129c97e160457
./../../music/Sage Francis/Still Sickly Business/08 Garden Gnomes.mp3 - 5e51ad883f2129c97e160458
./../../music/Sage Francis/Still Sickly Business/22 Whore Monger Freestyle (live).mp3 - 5e51ad923f2129c97e160459
./../../music/Sage Francis/Still Sickly Business/04 My Head.mp3 - 5e51ada43f2129c97e16045a
./../../music/Sage Francis/Still Sickly Business/13 Eye of the Tiger.mp3 - 5e51adb33f2129c97e16045b
./../../music/Sage Francis/Still Sickly Business/23 Whore Monger.mp3 - 5e51adc53f2129c97e16045c
./../../music/Sage Francis/Still Sickly Business/17 Pen to the Gun Fight (live).mp3 - 5e51adcd3f2129c

./../../music/Sage Francis/Personal Journals/03 Personal Journalist.mp3 - 5e51b20b3f2129c97e1604a2
./../../music/Sage Francis/Personal Journals/05 Climb Trees.mp3 - 5e51b21b3f2129c97e1604a3
./../../music/Sage Francis/Personal Journals/06 Broken Wings.mp3 - 5e51b22c3f2129c97e1604a4
./../../music/Sage Francis/The Makeshift Patriot EP/03 Makeshift Patriot.mp3 - 5e51b2463f2129c97e1604a5
./../../music/Sage Francis/The Makeshift Patriot EP/05 Every Midnight.mp3 - 5e51b25d3f2129c97e1604a6
./../../music/Sage Francis/The Makeshift Patriot EP/06 Hey Bobby.mp3 - 5e51b26f3f2129c97e1604a7
./../../music/Sage Francis/The Makeshift Patriot EP/02 Mourning Aftermath.mp3 - 5e51b2843f2129c97e1604a8
./../../music/Sage Francis/The Makeshift Patriot EP/01 Narcissist.mp3 - 5e51b2943f2129c97e1604a9
./../../music/Sage Francis/The Makeshift Patriot EP/04 Hangtime.mp3 - 5e51b2a73f2129c97e1604aa
./../../music/Sage Francis/The Known Unsoldier_ _Sick of Waging War_/11 Embarrassed.mp3 - 5e51b2b73f2129c97e1604ab
./../

./../../music/Jimi Hendrix/Voodoo Child_ The Jimi Hendrix Collection/25 Johnny B. Goode.mp3 - 5e51b7063f2129c97e1604ec
./../../music/Jimi Hendrix/Voodoo Child_ The Jimi Hendrix Collection/09 All Along the Watchtower.mp3 - 5e51b71b3f2129c97e1604ed
./../../music/Jimi Hendrix/Voodoo Child_ The Jimi Hendrix Collection/28 Purple Haze.mp3 - 5e51b7303f2129c97e1604ee
./../../music/Jimi Hendrix/Voodoo Child_ The Jimi Hendrix Collection/29 Star Spangled Banner.mp3 - 5e51b7423f2129c97e1604ef
./../../music/Jimi Hendrix/Voodoo Child_ The Jimi Hendrix Collection/19 Fire.mp3 - 5e51b7543f2129c97e1604f0
./../../music/Jimi Hendrix/Voodoo Child_ The Jimi Hendrix Collection/13 Stone Free.mp3 - 5e51b7673f2129c97e1604f1
./../../music/Jimi Hendrix/Voodoo Child_ The Jimi Hendrix Collection/04 Fire.mp3 - 5e51b7743f2129c97e1604f2
./../../music/Jimi Hendrix/Voodoo Child_ The Jimi Hendrix Collection/20 Hey Joe.mp3 - 5e51b7913f2129c97e1604f3
./../../music/Jimi Hendrix/Voodoo Child_ The Jimi Hendrix Collection/26 R

./../../music/A Tribe Called Quest/We Got It From Here… Thank You 4 Your Service/06 Kids.._.m4a - 5e51bd8c3f2129c97e16053d
./../../music/A Tribe Called Quest/We Got It From Here… Thank You 4 Your Service/12 Lost Somebody.m4a - 5e51bda13f2129c97e16053e
./../../music/A Tribe Called Quest/The Anthology/01 Check the Rhime.mp3 - 5e51bdb13f2129c97e16053f
./../../music/A Tribe Called Quest/The Anthology/15 Description of a Fool.mp3 - 5e51bdcb3f2129c97e160540
./../../music/A Tribe Called Quest/The Anthology/12 Oh My God.mp3 - 5e51bddc3f2129c97e160541
./../../music/A Tribe Called Quest/The Anthology/16 Keeping It Moving.mp3 - 5e51bdef3f2129c97e160542
./../../music/A Tribe Called Quest/The Anthology/18 Sucka Nigga.mp3 - 5e51be003f2129c97e160543
./../../music/A Tribe Called Quest/The Anthology/03 Award Tour.mp3 - 5e51be103f2129c97e160544
./../../music/A Tribe Called Quest/The Anthology/04 Can I Kick It_.mp3 - 5e51be253f2129c97e160545
./../../music/A Tribe Called Quest/The Anthology/19 Vivrant Thi

./../../music/Ratatat/LP3/04 Mirando.mp3 - 5e51c4653f2129c97e16059c
./../../music/Ratatat/LP3/12 Gipsy Threat.mp3 - 5e51c46e3f2129c97e16059d
./../../music/Ratatat/LP3/11 Mumtaz Khan.mp3 - 5e51c47a3f2129c97e16059e
./../../music/Ratatat/LP3/10 Bruleé.mp3 - 5e51c48a3f2129c97e16059f
./../../music/Ratatat/LP3/07 Shempi.mp3 - 5e51c49b3f2129c97e1605a0
./../../music/Ratatat/LP3/06 Bird Priest.mp3 - 5e51c4a93f2129c97e1605a1
./../../music/Ratatat/LP3/13 Black Heroes.mp3 - 5e51c4ba3f2129c97e1605a2
./../../music/Ratatat/LP3/02 Falcon Jab.mp3 - 5e51c4cb3f2129c97e1605a3
./../../music/Ratatat/LP3/09 Dura.mp3 - 5e51c4d93f2129c97e1605a4
./../../music/Ratatat/LP3/05 Flynn.mp3 - 5e51c4e33f2129c97e1605a5
./../../music/Ratatat/LP3/08 Imperials.mp3 - 5e51c4f23f2129c97e1605a6
./../../music/Ratatat/LP3/01 Shiller.mp3 - 5e51c5043f2129c97e1605a7
./../../music/Ratatat/LP3/03 Mi Viejo.mp3 - 5e51c5103f2129c97e1605a8
./../../music/Ratatat/9 Beats/07 Seven.mp3 - 5e51c51a3f2129c97e1605a9
./../../music/Ratatat/9 Beat